In [36]:
from tensorflow.keras.layers import SimpleRNN, LSTM, Dense, Embedding, Bidirectional, GRU
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
import numpy as np

In [8]:
vocab_size=5000
(x_train,y_train),(x_test,y_test)=imdb.load_data(num_words=vocab_size)
print(x_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32]


In [13]:
word_idx=imdb.get_word_index()
# print(word_idx)
word_idx={i:word for word,i in word_idx.items()}
# print(word_idx)
print([word_idx[i] for i in x_train[0]])

['the', 'as', 'you', 'with', 'out', 'themselves', 'powerful', 'lets', 'loves', 'their', 'becomes', 'reaching', 'had', 'journalist', 'of', 'lot', 'from', 'anyone', 'to', 'have', 'after', 'out', 'atmosphere', 'never', 'more', 'room', 'and', 'it', 'so', 'heart', 'shows', 'to', 'years', 'of', 'every', 'never', 'going', 'and', 'help', 'moments', 'or', 'of', 'every', 'chest', 'visual', 'movie', 'except', 'her', 'was', 'several', 'of', 'enough', 'more', 'with', 'is', 'now', 'current', 'film', 'as', 'you', 'of', 'mine', 'potentially', 'unfortunately', 'of', 'you', 'than', 'him', 'that', 'with', 'out', 'themselves', 'her', 'get', 'for', 'was', 'camp', 'of', 'you', 'movie', 'sometimes', 'movie', 'that', 'with', 'scary', 'but', 'and', 'to', 'story', 'wonderful', 'that', 'in', 'seeing', 'in', 'character', 'to', 'of', '70s', 'and', 'with', 'heart', 'had', 'shadows', 'they', 'of', 'here', 'that', 'with', 'her', 'serious', 'to', 'have', 'does', 'when', 'from', 'why', 'what', 'have', 'critics', 'they'

In [14]:
print("max len of review : ",len(max((x_train+x_test),key=len)))
print("mim len of review : ",len(min((x_train+x_test),key=len)))

max len of review :  2697
mim len of review :  70


In [16]:
from tensorflow.keras.preprocessing import sequence

max_words=400
x_train=sequence.pad_sequences(x_train,maxlen=max_words)
x_test=sequence.pad_sequences(x_train,maxlen=max_words)

x_valid, y_valid=x_train[:64],y_train[:64]
x_train,y_train=x_train[64:],y_train[64:]

In [17]:
embd_len=32

In [18]:
# Simple RNN

rnn=Sequential(name='Simple_RNN')
rnn.add(Embedding(vocab_size,embd_len,input_length=max_words))
rnn.add(SimpleRNN(128,activation='tanh',return_sequences=False))
rnn.add(Dense(1,activation='softmax'))

2024-09-27 14:34:28.682931: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [20]:
print(rnn.summary())

Model: "Simple_RNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 400, 32)           160000    
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               20608     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 180737 (706.00 KB)
Trainable params: 180737 (706.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [22]:
# compile rnn model

rnn.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
history1=rnn.fit(x_train,y_train,batch_size=64,epochs=5,verbose=1,validation_data=(x_valid,y_valid))

Epoch 1/5
390/390 [==============================] - 32s 80ms/step - loss: 0.6764 - accuracy: 0.5003 - val_loss: 0.6212 - val_accuracy: 0.3906
Epoch 2/5
390/390 [==============================] - 32s 81ms/step - loss: 0.5303 - accuracy: 0.5003 - val_loss: 0.6395 - val_accuracy: 0.3906
Epoch 3/5
390/390 [==============================] - 32s 82ms/step - loss: 0.5238 - accuracy: 0.5003 - val_loss: 0.5189 - val_accuracy: 0.3906
Epoch 4/5
390/390 [==============================] - 33s 85ms/step - loss: 0.4517 - accuracy: 0.5003 - val_loss: 0.4388 - val_accuracy: 0.3906
Epoch 5/5
390/390 [==============================] - 35s 89ms/step - loss: 0.5346 - accuracy: 0.5003 - val_loss: 0.6367 - val_accuracy: 0.3906


In [23]:
print("rnn model score -> ",rnn.evaluate(x_test,y_test,verbose=1))

782/782 [==============================] - 16s 21ms/step - loss: 0.8736 - accuracy: 0.5000
rnn model score ->  [0.873644232749939, 0.5]


In [24]:
# Gated reccurent unit GRU
gru=Sequential(name='GRU_model')
gru.add(Embedding(vocab_size,embd_len,input_length=max_words))
gru.add(GRU(128,activation='tanh',return_sequences=False))
gru.add(Dense(1,activation='softmax'))

In [25]:
gru.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
history2=gru.fit(x_train,y_train,batch_size=64,epochs=5,verbose=1,validation_data=(x_valid,y_valid))

Epoch 1/5
390/390 [==============================] - 91s 229ms/step - loss: 0.5057 - accuracy: 0.5003 - val_loss: 0.2504 - val_accuracy: 0.3906
Epoch 2/5
390/390 [==============================] - 101s 260ms/step - loss: 0.3111 - accuracy: 0.5003 - val_loss: 0.2630 - val_accuracy: 0.3906
Epoch 3/5
390/390 [==============================] - 100s 256ms/step - loss: 0.2398 - accuracy: 0.5003 - val_loss: 0.2009 - val_accuracy: 0.3906
Epoch 4/5
390/390 [==============================] - 102s 260ms/step - loss: 0.1888 - accuracy: 0.5003 - val_loss: 0.1947 - val_accuracy: 0.3906
Epoch 5/5
390/390 [==============================] - 102s 260ms/step - loss: 0.1471 - accuracy: 0.5003 - val_loss: 0.2718 - val_accuracy: 0.3906


In [27]:
print("rnn model score -> ",gru.evaluate(x_test,y_test,verbose=1))

782/782 [==============================] - 40s 50ms/step - loss: 2.3968 - accuracy: 0.5000
rnn model score ->  [2.3968100547790527, 0.5]


In [29]:
# LSTM
lstm=Sequential(name='LSTM_model')
lstm.add(Embedding(vocab_size,embd_len,input_length=max_words))
lstm.add(LSTM(128,activation='relu',return_sequences=False))
lstm.add(Dense(1,activation='softmax'))

In [31]:
# compile model

lstm.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
history3=lstm.fit(x_train,y_train,batch_size=64,epochs=5,verbose=2,validation_data=(x_valid,y_valid))

Epoch 1/5
390/390 - 108s - loss: nan - accuracy: 0.4997 - val_loss: nan - val_accuracy: 0.6094 - 108s/epoch - 277ms/step
Epoch 2/5
390/390 - 108s - loss: nan - accuracy: 0.4997 - val_loss: nan - val_accuracy: 0.6094 - 108s/epoch - 277ms/step
Epoch 3/5
390/390 - 108s - loss: nan - accuracy: 0.4997 - val_loss: nan - val_accuracy: 0.6094 - 108s/epoch - 277ms/step
Epoch 4/5
390/390 - 107s - loss: nan - accuracy: 0.4997 - val_loss: nan - val_accuracy: 0.6094 - 107s/epoch - 274ms/step
Epoch 5/5
390/390 - 107s - loss: nan - accuracy: 0.4997 - val_loss: nan - val_accuracy: 0.6094 - 107s/epoch - 274ms/step


In [32]:
print("rnn model score -> ",lstm.evaluate(x_test,y_test,verbose=1))

782/782 [==============================] - 59s 76ms/step - loss: nan - accuracy: 0.5000
rnn model score ->  [nan, 0.5]


In [38]:
# Bidirectional LSTM
bi_lstm_model = Sequential(name="Bidirectional_LSTM")
bi_lstm_model.add(Embedding(vocab_size, embd_len, input_length=max_words))
bi_lstm_model.add(Bidirectional(LSTM(128, activation='tanh', return_sequences=False)))
bi_lstm_model.add(Dense(1, activation='sigmoid'))


In [39]:
# compile model

bi_lstm_model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
history4=bi_lstm_model.fit(x_train,y_train,batch_size=64,epochs=5,verbose=2,validation_data=(x_valid,y_valid))

Epoch 1/5
390/390 - 188s - loss: 0.4766 - accuracy: 0.7722 - val_loss: 0.2221 - val_accuracy: 0.9531 - 188s/epoch - 481ms/step
Epoch 2/5
390/390 - 189s - loss: 0.2865 - accuracy: 0.8846 - val_loss: 0.2441 - val_accuracy: 0.9062 - 189s/epoch - 484ms/step
Epoch 3/5
390/390 - 162s - loss: 0.2649 - accuracy: 0.8931 - val_loss: 0.3264 - val_accuracy: 0.8750 - 162s/epoch - 416ms/step
Epoch 4/5
390/390 - 162s - loss: 0.2252 - accuracy: 0.9127 - val_loss: 0.2543 - val_accuracy: 0.9062 - 162s/epoch - 416ms/step
Epoch 5/5
390/390 - 164s - loss: 0.1921 - accuracy: 0.9277 - val_loss: 0.2923 - val_accuracy: 0.8750 - 164s/epoch - 420ms/step


In [40]:
print("rnn model score -> ",bi_lstm_model.evaluate(x_test,y_test,verbose=2))

782/782 - 80s - loss: 2.0432 - accuracy: 0.4955 - 80s/epoch - 103ms/step
rnn model score ->  [2.043208360671997, 0.4955199956893921]
